In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
import numpy as np
import gzip,csv


In [2]:
train = pd.read_csv(r'train.csv')
y= train.Category.values
train = train.drop(['Address','Category','Dates','Descript','X','Y','Resolution'],axis=1)


In [3]:
days = {}
cnt=0
for i in np.unique(train.DayOfWeek.values):
    days[i] = cnt
    cnt+=1


In [4]:
dict ={'DayOfWeek' : days}

In [5]:
PdDis = {}
cnt=0
for i in np.unique(train.PdDistrict.values):
    PdDis[i] = cnt
    cnt+=1


In [6]:
dict['PdDistrict'] = PdDis

In [7]:
train = train.replace(dict)

In [8]:
#model = BernoulliNB()
model = GaussianNB()
model.fit(train,y)


GaussianNB()

In [9]:
test = pd.read_csv(r'test.csv')
idx = test.Id.values
test = test.drop(['Id','Dates','Address','X','Y'],axis=1)



In [10]:
test = test.replace(dict)


In [11]:
predicted = np.array(model.predict_proba(test))
labels = ['Id']
for i in model.classes_:
    labels.append(i)
with gzip.open('bernoulinb.csv.gz', 'w') as outf:
  fo = csv.writer(outf, lineterminator='\n')
  fo.writerow(labels)

  for i, pred in enumerate(predicted):
    fo.writerow([i] + list(pred))


In [12]:
#EDA

In [13]:
import zipfile
import matplotlib.pyplot as pl


In [15]:
z = zipfile.ZipFile('train.csv.zip')
print(z.namelist())


['train.csv']


In [16]:
train = pd.read_csv(z.open('train.csv'), parse_dates=['Dates'])

In [17]:
train['Year'] = train['Dates'].map(lambda x: x.year)
train['Week'] = train['Dates'].map(lambda x: x.week)
train['Hour'] = train['Dates'].map(lambda x: x.hour)


In [18]:
print(train.head())

                Dates        Category                      Descript  \
0 2015-05-13 23:53:00        WARRANTS                WARRANT ARREST   
1 2015-05-13 23:53:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
2 2015-05-13 23:33:00  OTHER OFFENSES      TRAFFIC VIOLATION ARREST   
3 2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   
4 2015-05-13 23:30:00   LARCENY/THEFT  GRAND THEFT FROM LOCKED AUTO   

   DayOfWeek PdDistrict      Resolution                    Address  \
0  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
1  Wednesday   NORTHERN  ARREST, BOOKED         OAK ST / LAGUNA ST   
2  Wednesday   NORTHERN  ARREST, BOOKED  VANNESS AV / GREENWICH ST   
3  Wednesday   NORTHERN            NONE   1500 Block of LOMBARD ST   
4  Wednesday       PARK            NONE  100 Block of BRODERICK ST   

            X          Y  Year  Week  Hour  
0 -122.425892  37.774599  2015    20    23  
1 -122.425892  37.774599  2015    20    23  
2 -122.424363  37

In [19]:
train.PdDistrict.value_counts().plot(kind='bar', figsize=(8,10))
pl.savefig('district_counts.png')


In [20]:
train['event']=1
weekly_events = train[['Week','Year','event']].groupby(['Year','Week']).count().reset_index()
weekly_events_years = weekly_events.pivot(index='Week', columns='Year', values='event').fillna(method='ffill')
#%matplotlib inline
ax = weekly_events_years.interpolate().plot(title='number of cases every 2 weeks', figsize=(10,6))
pl.savefig('events_every_two_weeks.png')

In [21]:
hourly_events = train[['Hour','event']].groupby(['Hour']).count().reset_index()
hourly_events.plot(kind='bar', figsize=(6, 6))
pl.savefig('hourly_events.png')


In [22]:
hourly_district_events = train[['PdDistrict','Hour','event']].groupby(['PdDistrict','Hour']).count().reset_index()
hourly_district_events_pivot = hourly_district_events.pivot(index='Hour', columns='PdDistrict', values='event').fillna(method='ffill')
hourly_district_events_pivot.interpolate().plot(title='number of cases hourly by district', figsize=(10,6))
pl.savefig('hourly_events_by_district.png')